In [ ]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
from io import StringIO

In [ ]:
load_dotenv('../.env')

In [ ]:
context = session.load_context()
credentials = context._get_config_credentials()

In [ ]:
def get_months(start:str, end:str):

    start=pd.Timestamp(start)
    end=pd.Timestamp(end)
    months = pd.date_range(start=start, end=end, freq='M')
    
    if len(months) > 0:
    
        if months[0] > start:
            months = months.insert(0,start)
    
        if months[-1] < end:
            months = months.append(pd.DatetimeIndex([end]))
        
    else:
        months = pd.DatetimeIndex([start,end])
        
    return months
    

In [ ]:
months = get_months(start='2022-01-01', end='2022-10-18')
months

In [ ]:
months_str = [str(date) for date in months.date]

In [ ]:
months_str

In [ ]:
def read_month(start:str, end:str, session:requests.Session):
    
    url = f'https://online.blueflow.se/be/external/raw/2?fromDate={start}&toDate={end}'
    r = session.get(url)
    s = r.content.decode('utf-8')
    df = pd.read_csv(StringIO(s))
    return df

In [ ]:
def load_blueflow(start:str, end:str, user:str, password:str):
    
    months = get_months(start=start, end=end)
    months_str = [str(date) for date in months.date]
    
    files = {}
    
    with requests.Session() as session:
        session.auth = (user, password)
        
        for i,month_str in enumerate(months_str[0:-1]):
            start_month = month_str
            end_month = months_str[i+1]
            s_month = read_month(start=start_month, end=end_month, session=session)
            
            file_name = f'{start_month}--{end_month}'
            files[file_name] = s_month
            
    return files


In [ ]:
files = load_blueflow(start='2022-09-01', end='2022-10-18', user=credentials['user'], password=credentials['pass'])

In [ ]:
files.keys()